In [2]:
import openai
# from openai.embeddings_utils import get_embeddings, cosine_similarity

In [3]:
import pandas as pd
import os
import re
import time
import numpy as np

In [4]:
pd.options.display.float_format = '{:.4f}'.format

In [5]:
with open(os.path.join(os.path.dirname(os.path.abspath("")), "OpenAI_API_my.txt"), 'r') as f:
    openai.api_key = f.readlines()[0]

In [6]:
embedding_model = "text-embedding-3-small"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

completion_model = "gpt-3.5-turbo"

In [7]:
def get_embedding(text: str, model="text-embedding-3-small", **kwargs):
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")
    response = openai.embeddings.create(input=[text], model=model, **kwargs)
    return response.data[0].embedding

In [8]:
def calc_cos_sim(vector_a, vector_b):

    # Compute the dot product of the two vectors
    dot_product = np.dot(vector_a, vector_b)
    
    # Compute the L2 norms (magnitudes) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    
    # Compute the cosine similarity
    # Note: We add a small epsilon value to the denominator for numerical stability
    epsilon = 1e-10
    cosine_similarity = dot_product / (norm_a * norm_b + epsilon)
    
    return cosine_similarity

In [15]:
embeddings = get_embedding("yellow red apple on tree branch")

In [16]:
len(embeddings)

1536

In [9]:
train_languages = ['English', 'Spanish', 'French', 'German', 'Italian', 'Dutch', 'Russian', 'Chinese',
'Japanese', 'Korean', 'Arabic', 'Portuguese', 'Swedish', 'Norwegian', 'Danish', 'Finnish', 'Hindi', 'Turkish', 'Greek']

In [10]:
len(train_languages)

19

In [18]:
test_languages = ['Polish', 'Czech', 'Ukrainian', 'Hebrew']

In [19]:
client = openai.OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

In [20]:
def chatgpt(text, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(model=model,
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": text},
        ],
        temperature=0)
    return response.choices[0].message.content

In [21]:
text_data = pd.read_csv('./data/text_data.csv', encoding='utf-8')

In [39]:
text_data

,themes,English1,English2,English3,Spanish1,Spanish2,Spanish3,French1,French2,French3,...,Polish3,Czech1,Czech2,Czech3,Ukrainian1,Ukrainian2,Ukrainian3,Hebrew1,Hebrew2,Hebrew3
0,Love and Relationships,Love is a beautiful and complex emotion that h...,Relationships thrive when there is open and ho...,Love knows no boundaries and can be found in v...,El amor es una emoción hermosa y compleja que ...,Las relaciones prosperan cuando hay una comuni...,El amor no conoce fronteras y puede encontrars...,L'amour est une émotion belle et complexe qui ...,Les relations s'épanouissent lorsque la commun...,L'amour ne connaît pas de frontières et peut ê...,...,Miłość nie zna granic i może przybierać różne ...,"Láska je krásná a složitá emoce, která má moc ...","Vztahy prosperují, když existuje otevřená a up...",Láska nezná hranice a může se objevit v různýc...,"Любов - це прекрасна і складна емоція, яка має...","Відносини процвітають, коли є відкрита і чесна...",Любов не знає меж і може бути знайдена у різни...,אהבה היא רגש יפה ומורכב שיש לו את הכוח להביא ש...,"היחסים מתפתחים כשיש תקשורת פתוחה וכנה, כבוד מש...",אהבה אינה מכירה גבולות וניתן למצואה בצורות שונ...
1,Nature and the Environment,The lush green forests are not only a sight to...,The delicate balance of ecosystems is being di...,It is our responsibility to protect and preser...,Los exuberantes bosques verdes no solo son un ...,El delicado equilibrio de los ecosistemas está...,Es nuestra responsabilidad proteger y preserva...,Les forêts luxuriantes et verdoyantes ne sont ...,Le fragile équilibre des écosystèmes est pertu...,Il est de notre responsabilité de protéger et ...,...,Naszym obowiązkiem jest chronić i zachować prz...,"Husté zelené lesy nejsou jen pohledem, ale tak...",Jemné rovnováhy ekosystémů jsou narušovány lid...,Je naší odpovědností chránit a zachovávat přír...,"Буйні зелені ліси - це не тільки вид, який вар...",Ніжна рівновага екосистем порушується людською...,Нашою відповідальністю є захист і збереження п...,היערות הירוקים והצומחים אינם רק נוף נפלא להביט...,האיזון העדין של האקוסיסטמות מופר בפעילויות אנו...,זהו עלינו להגן ולשמור על הטבע למען הדורות הבאי...
2,Technology and the Future,With advancements in artificial intelligence a...,As we continue to explore the possibilities of...,The rapid development of virtual reality and a...,Con los avances en inteligencia artificial y a...,A medida que continuamos explorando las posibi...,El rápido desarrollo de las tecnologías de rea...,Avec les avancées de l'intelligence artificiel...,Alors que nous continuons à explorer les possi...,Le développement rapide des technologies de ré...,...,Szybki rozwój technologii rzeczywistości wirtu...,S rozvojem umělé inteligence a strojového učen...,Pokračujeme v prozkoumávání možností obnovitel...,Rychlý vývoj technologií virtuální reality a r...,Завдяки досягненням у сфері штучного інтелекту...,Поки ми продовжуємо досліджувати можливості ві...,Швидкий розвиток технологій віртуальної реальн...,"עם התקדמות בבינה מלאכותית ולמידת מכונה, עתיד ה...",כך שאנחנו ממשיכים לחקור את האפשרויות של מקורות...,התפתחות מהירה של טכנולוגיות מציאות מדומה ומציא...
3,Identity and Self-Discovery,Self-discovery is a lifelong journey that allo...,"Exploring different cultures, beliefs, and exp...",The process of self-discovery often involves q...,El autodescubrimiento es un viaje de toda la v...,"Explorar diferentes culturas, creencias y expe...",El proceso de autodescubrimiento a menudo impl...,La découverte de soi est un voyage tout au lon...,"Explorer différentes cultures, croyances et ex...",Le processus de découverte de soi implique sou...,...,Proces odkrywania samego siebie często wiąże s...,"Sebepoznání je celoživotní cesta, která jednot...","Prozkoumávání různých kultur, přesvědčení a zk...",Proces sebepoznání často zahrnuje otázky k soc...,"Самовідкриття - це життєва подорож, яка дозвол...","Дослідження різних культур, вірувань та досвід...",Процес самовідкриття часто включає у себе пита...,גילוי עצמי הוא מ

In [35]:
text_data1 = text_data.copy()

# Load same texts and questions in "train languages"

# Get embeddings

In [138]:
embd_np = np.load('./data/embd_data.npy')
embd_np.shape

(21, 69, 1536)

# Extract english entity from train texts

In [66]:
embd_np[0][0]

array([ 0.02332043, -0.0065157 ,  0.00080158, ...,  0.02321733,
       -0.0099385 ,  0.01359843])

### Extracting meanings of sentences. First sentence is test

In [256]:
meanings = np.zeros((21, 3, 1536))
for i in range(int(embd_np.shape[1] / 3)):
    meanings += embd_np[:, i * 3: (i+1) * 3]

meanings /= int(embd_np.shape[1] / 3)

In [257]:
meanings.shape

(21, 3, 1536)

In [258]:
list(text_data.columns).index('Hebrew1')

67

In [259]:
# save all entities
folder = './entities/'
for i, lang in enumerate(train_languages + test_languages):
    entity_arr = embd_np[:, i * 3: i * 3 + 3, :] - meanings
    entity = entity_arr.mean(0).mean(0)
    np.save(folder + lang.lower() +'.npy', entity)

In [188]:
eng_entity_arr = embd_np[1:, :3, :] - meanings
fr_entity_arr = embd_np[1:, 6:9, :] - meanings
ch_entity_arr = embd_np[1:, 21:24, :] - meanings
heb_entity_arr = embd_np[1:, 66:69, :] - meanings

In [189]:
eng_entity_arr.std(0).std(0).max()

0.005494653480927692

In [190]:
eng_entity = eng_entity_arr.mean(0).mean(0)
fr_entity = fr_entity_arr.mean(0).mean(0)
ch_entity = ch_entity_arr.mean(0).mean(0)
heb_entity = heb_entity_arr.mean(0).mean(0)

In [74]:
np.save('./entities/english.npy', eng_entity)
np.save('./entities/hebrew.npy', heb_entity)
np.save('./entities/french.npy', fr_entity)
np.save('./entities/chinese.npy', ch_entity)

In [191]:
eng_entity.shape

(1536,)

## Compare mean entity vector with trained by linear regression

In [247]:
eng_entity_trained = np.load('./entities/english2.npy')

In [248]:
eng_entity_diff = eng_entity - eng_entity_trained

In [250]:
eng_entity_diff.min()

-0.9900089265892555

---

In [189]:
for i in range(len(embd_data.columns)):
    print(i, embd_data.columns[i])

0 English1
1 English2
2 English3
3 Spanish1
4 Spanish2
5 Spanish3
6 French1
7 French2
8 French3
9 German1
10 German2
11 German3
12 Italian1
13 Italian2
14 Italian3
15 Dutch1
16 Dutch2
17 Dutch3
18 Russian1
19 Russian2
20 Russian3
21 Chinese1
22 Chinese2
23 Chinese3
24 Japanese1
25 Japanese2
26 Japanese3
27 Korean1
28 Korean2
29 Korean3
30 Arabic1
31 Arabic2
32 Arabic3
33 Portuguese1
34 Portuguese2
35 Portuguese3
36 Swedish1
37 Swedish2
38 Swedish3
39 Norwegian1
40 Norwegian2
41 Norwegian3
42 Danish1
43 Danish2
44 Danish3
45 Finnish1
46 Finnish2
47 Finnish3
48 Hindi1
49 Hindi2
50 Hindi3
51 Turkish1
52 Turkish2
53 Turkish3
54 Greek1
55 Greek2
56 Greek3
57 Polish1
58 Polish2
59 Polish3
60 Czech1
61 Czech2
62 Czech3
63 Ukrainian1
64 Ukrainian2
65 Ukrainian3
66 Hebrew1
67 Hebrew2
68 Hebrew3


# Search results from test languages

In [76]:
ques_eng1 = []
for txt in text_data["English1"]:
    ques_eng1.append(chatgpt(f'Ask question on this sentence: {txt}'))

In [77]:
ques_eng1_embd = []
for ques in ques_eng1:
    ques_eng1_embd.append(get_embedding(ques))
ques_eng1_embd = np.array(ques_eng1_embd)

In [192]:
len(ques_eng1)

21

## Cos similarity between: question to first sentece in each theme and all 3 sentences for each theme

In [237]:
q1_eng_fr_ch = pd.DataFrame(columns=list(embd_data.columns[:3]) + list(embd_data.columns[6:9]) + list(embd_data.columns[21:24]))

In [238]:
id_columns = list(embd_data.columns)
for r in range(embd_np.shape[0]):
    row_data = []
    for col in q1_eng_fr_ch.columns:
        c = id_columns.index(col)
        row_data.append(calc_cos_sim(ques_eng1_embd[r], embd_np[r, c]))
    q1_eng_fr_ch.loc[r] = row_data

In [239]:
q1_eng_fr_ch[' |'] = ['  ' for _ in range(embd_np.shape[0])]
q1_eng_fr_ch['| '] = ['  ' for _ in range(embd_np.shape[0])]

In [240]:
q1_eng_fr_ch = q1_eng_fr_ch[['English1','French1','Chinese1',' |','English2','French2','Chinese2','| ','English3','French3', 'Chinese3']].round(decimals=4)

In [241]:
q1_eng_fr_ch

,English1,French1,Chinese1,|,English2,French2,Chinese2,|,English3,French3,Chinese3
0,0.6396,0.5084,0.5188,,0.5283,0.3939,0.4068,,0.4142,0.3336,0.3911
1,0.6838,0.5641,0.5148,,0.4503,0.3765,0.3774,,0.4789,0.3808,0.4060
2,0.8579,0.6841,0.6518,,0.3140,0.2264,0.2280,,0.4172,0.3862,0.3795
3,0.8332,0.6013,0.5510,,0.4642,0.3972,0.4413,,0.6887,0.5092,0.4667
4,0.6766,0.5681,0.5489,,0.4839,0.3980,0.3405,,0.4283,0.3250,0.3149
5,0.7213,0.5993,0.5658,,0.6434,0.5043,0.5367,,0.6725,0.5319,0.5045
6,0.5346,0.4110,0.3518,,0.3714,0.2733,0.2762,,0.2145,0.1321,0.2226
7,0.5967,0.4720,0.4363,,0.4129,0.3461,0.3178,,0.4980,0.3607,0.4014
8,0.8526,0.6570,0.6659,,0.4920,0.3676,0.4060,,0.4490,0.3450,0.3881
9,0.7204,0.5151,0.4747,,0.4932,0.3603,0.3816,,0.5523,0.4253,0.4217


In [184]:
print("0.7529	0.6473	0.5155		0.3265	0.2924	0.2749		0.2463	0.1344	0.1358")

0.7529	0.6473	0.5155		0.3265	0.2924	0.2749		0.2463	0.1344	0.1358


## Now we removing language entity from sentence embeddings
    Cos similarity between: question to first sentece in each theme and all 3 sentences for each theme

In [251]:
q1_eng_fr_ch_nolang = pd.DataFrame(columns=list(embd_data.columns[:3]) + list(embd_data.columns[6:9]) + list(embd_data.columns[21:24]))

In [252]:
id_columns = list(embd_data.columns)
lang_entity = {'English': eng_entity_trained, 'French': fr_entity, 'Chinese': ch_entity}
for r in range(embd_np.shape[0]):
    row_data = []
    for col in q1_eng_fr_ch_nolang.columns:
        c = id_columns.index(col)
        row_data.append(calc_cos_sim(ques_eng1_embd[r], embd_np[r, c] - lang_entity[col[:-1]]))
    q1_eng_fr_ch_nolang.loc[r] = row_data

In [253]:
q1_eng_fr_ch_nolang[' |'] = ['  ' for _ in range(embd_np.shape[0])]
q1_eng_fr_ch_nolang['| '] = ['  ' for _ in range(embd_np.shape[0])]

In [254]:
q1_eng_fr_ch_nolang = q1_eng_fr_ch_nolang[['English1','French1','Chinese1',' |','English2','French2','Chinese2','| ','English3','French3', 'Chinese3']].round(decimals=4)

In [255]:
q1_eng_fr_ch_nolang

,English1,French1,Chinese1,|,English2,French2,Chinese2,|,English3,French3,Chinese3
0,-0.0054,0.5461,0.5287,,-0.0101,0.4235,0.4135,,-0.0149,0.3484,0.3946
1,-0.0429,0.5771,0.5184,,-0.0527,0.3757,0.3677,,-0.0514,0.3868,0.4061
2,-0.0214,0.7186,0.6655,,-0.0442,0.2361,0.2296,,-0.0399,0.3997,0.3782
3,-0.0363,0.6298,0.5673,,-0.0518,0.4005,0.4466,,-0.0424,0.5276,0.4685
4,-0.0339,0.6102,0.5674,,-0.0419,0.4147,0.3436,,-0.0443,0.3417,0.3147
5,-0.0467,0.6178,0.5692,,-0.0499,0.5271,0.5441,,-0.0487,0.5594,0.5045
6,-0.0271,0.4440,0.3460,,-0.0340,0.3000,0.2630,,-0.0405,0.1507,0.2127
7,-0.0697,0.4997,0.4352,,-0.0774,0.3600,0.3006,,-0.0739,0.3772,0.3938
8,-0.0402,0.6893,0.6767,,-0.0553,0.3753,0.4014,,-0.0571,0.3471,0.3736
9,-0.0567,0.5535,0.4771,,-0.0662,0.3943,0.3663,,-0.0637,0.4593,0.4134


In [183]:
print("0.7134	0.6666	0.5178		0.2975	0.3020	0.2792		0.2125	0.1366	0.1353")

0.7134	0.6666	0.5178		0.2975	0.3020	0.2792		0.2125	0.1366	0.1353


## Lets ask 1 question to all sentences (language entities not removed)

In [203]:
question_num = 0

In [204]:
q1_eng_fr_ch_specq = pd.DataFrame(columns=list(embd_data.columns[:3]) + list(embd_data.columns[6:9]) + list(embd_data.columns[21:24]))

In [205]:
id_columns = list(embd_data.columns)
for r in range(embd_np.shape[0]):
    row_data = []
    for col in q1_eng_fr_ch_specq.columns:
        c = id_columns.index(col)
        row_data.append(calc_cos_sim(ques_eng1_embd[question_num], embd_np[r, c]))
    q1_eng_fr_ch_specq.loc[r] = row_data

In [206]:
q1_eng_fr_ch_specq[' |'] = ['  ' for _ in range(embd_np.shape[0])]
q1_eng_fr_ch_specq['| '] = ['  ' for _ in range(embd_np.shape[0])]

In [207]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

In [208]:
q1_eng_fr_ch_specq = q1_eng_fr_ch_specq[['English1','French1','Chinese1',' |','English2','French2','Chinese2','| ','English3','French3', 'Chinese3']].round(decimals=4)

In [209]:
print('Red font - max value in column. Yellow bg - asked question\n')
q1_eng_fr_ch_specq.style.apply(highlight_max, props='color:red;', axis=0)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=pd.IndexSlice[question_num])

Red font - max value in column. Yellow bg - asked question



,English1,French1,Chinese1,|,English2,French2,Chinese2,|,English3,French3,Chinese3
0,0.639600,0.508400,0.518800,,0.528300,0.393900,0.406800,,0.414200,0.333600,0.391100
1,0.107900,0.072800,0.050300,,0.166100,0.080800,0.085600,,0.241300,0.181800,0.180800
2,0.117600,0.049700,0.061800,,0.136800,0.079800,0.080300,,0.095000,0.063300,0.057300
3,0.254500,0.135400,0.213400,,0.174800,0.123700,0.159100,,0.252800,0.166800,0.205800
4,0.163100,0.106200,0.137900,,0.247900,0.212200,0.165000,,0.250000,0.183800,0.201400
5,0.072200,0.034000,0.045100,,0.187200,0.109400,0.087100,,0.167600,0.114600,0.167100
6,0.184000,0.131400,0.129700,,0.160300,0.087300,0.063100,,0.195800,0.133600,0.178600
7,0.162100,0.117200,0.167100,,0.149200,0.096900,0.137200,,0.152800,0.109700,0.149600
8,0.200100,0.160600,0.202100,,0.208800,0.123700,0.175000,,0.196200,0.110700,0.155000
9,0.283300,0.227300,0.193200,,0.273400,0.224300,0.244700,,0.300100,0.252800,0.290600


In [210]:
print("0.0784	0.0380	0.0434		0.0904	0.0483	0.0520		0.1371	0.0883	0.0662")

0.0784	0.0380	0.0434		0.0904	0.0483	0.0520		0.1371	0.0883	0.0662


## Lets ask 1 question to all sentences (language entities are removed)

In [211]:
question_num = 5

In [212]:
q1_eng_fr_ch_nolang_specq = pd.DataFrame(columns=list(embd_data.columns[:3]) + list(embd_data.columns[6:9]) + list(embd_data.columns[21:24]))

In [213]:
id_columns = list(embd_data.columns)
lang_entity = {'English': eng_entity, 'French': fr_entity, 'Chinese': ch_entity}
for r in range(embd_np.shape[0]):
    row_data = []
    for col in q1_eng_fr_ch_nolang_specq.columns:
        c = id_columns.index(col)
        row_data.append(calc_cos_sim(ques_eng1_embd[question_num], embd_np[r, c] - lang_entity[col[:-1]]))
    q1_eng_fr_ch_nolang_specq.loc[r] = row_data

In [214]:
q1_eng_fr_ch_nolang_specq[' |'] = ['  ' for _ in range(embd_np.shape[0])]
q1_eng_fr_ch_nolang_specq['| '] = ['  ' for _ in range(embd_np.shape[0])]

In [215]:
q1_eng_fr_ch_nolang_specq = q1_eng_fr_ch_nolang_specq[['English1','French1','Chinese1',' |','English2','French2','Chinese2','| ','English3','French3', 'Chinese3']].round(decimals=4)

In [216]:
print('Red font - max value in column. Yellow bg - asked question\n')
q1_eng_fr_ch_nolang_specq.style.apply(highlight_max, props='color:red;', axis=0)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=pd.IndexSlice[question_num])

Red font - max value in column. Yellow bg - asked question



,English1,French1,Chinese1,|,English2,French2,Chinese2,|,English3,French3,Chinese3
0,0.107700,0.141700,0.168000,,0.092700,0.122900,0.111600,,0.142800,0.159500,0.194700
1,0.083100,0.131900,0.067500,,0.058700,0.060400,0.063800,,0.075100,0.093800,0.075500
2,0.149100,0.157400,0.191600,,0.127100,0.131000,0.155200,,0.203600,0.227600,0.163700
3,0.193900,0.223400,0.194800,,0.279100,0.289600,0.296200,,0.193100,0.191700,0.209400
4,0.075400,0.085900,0.056800,,0.027400,0.051800,0.033700,,0.092100,0.103500,0.089400
5,0.645800,0.617800,0.569200,,0.574700,0.527100,0.544100,,0.604600,0.559400,0.504500
6,0.109700,0.108500,0.085900,,-0.018500,0.016000,-0.009000,,-0.051400,-0.047300,-0.054900
7,0.181000,0.198500,0.193200,,0.083800,0.112600,0.091900,,0.214500,0.215700,0.226200
8,0.095400,0.094600,0.012800,,0.102800,0.118800,0.128700,,0.097800,0.059000,0.046200
9,0.094000,0.067800,0.126900,,0.011400,0.070400,0.046900,,0.133200,0.114800,0.096100


In [217]:
print("-0.0861	-0.0807	-0.0786		-0.0220	-0.0313	-0.0251		-0.0433	0.0085	-0.0316")

-0.0861	-0.0807	-0.0786		-0.0220	-0.0313	-0.0251		-0.0433	0.0085	-0.0316


## Other languages

In [218]:
q1_eng_pol_heb = pd.DataFrame(columns=list(embd_data.columns[:3]) + list(embd_data.columns[57:60]) + list(embd_data.columns[66:69]))

In [219]:
id_columns = list(embd_data.columns)
for r in range(embd_np.shape[0]):
    row_data = []
    for col in q1_eng_pol_heb.columns:
        c = id_columns.index(col)
        row_data.append(calc_cos_sim(ques_eng1_embd[r], embd_np[r, c]))
    q1_eng_pol_heb.loc[r] = row_data

In [220]:
q1_eng_pol_heb[' |'] = ['  ' for _ in range(embd_np.shape[0])]
q1_eng_pol_heb['| '] = ['  ' for _ in range(embd_np.shape[0])]

In [221]:
q1_eng_pol_heb = q1_eng_pol_heb[['English1','Polish1','Hebrew1',' |','English2','Polish2','Hebrew2','| ','English3','Polish3', 'Hebrew3']].round(decimals=4)

In [222]:
q1_eng_pol_heb

,English1,Polish1,Hebrew1,|,English2,Polish2,Hebrew2,|,English3,Polish3,Hebrew3
0,0.6396,0.4511,0.3095,,0.5283,0.3644,0.1330,,0.4142,0.2942,0.3248
1,0.6838,0.4721,0.1674,,0.4503,0.2952,0.0953,,0.4789,0.2995,0.1707
2,0.8579,0.5331,0.1968,,0.3140,0.1575,0.1793,,0.4172,0.2752,0.1119
3,0.8332,0.5004,0.2053,,0.4642,0.3780,0.1085,,0.6887,0.4434,0.2753
4,0.6766,0.4641,0.3249,,0.4839,0.2951,0.2832,,0.4283,0.2422,0.2322
5,0.7213,0.4950,0.2666,,0.6434,0.4576,0.2464,,0.6725,0.4389,0.2034
6,0.5346,0.2539,0.2728,,0.3714,0.1962,0.2033,,0.2145,0.1456,0.0892
7,0.5967,0.3715,0.1558,,0.4129,0.2102,0.1482,,0.4980,0.3372,0.0883
8,0.8526,0.5592,0.1379,,0.4920,0.3340,-0.0148,,0.4490,0.3420,0.0570
9,0.7204,0.3832,0.2083,,0.4932,0.1402,0.1202,,0.5523,0.3245,0.1167


In [179]:
print("0.7529	0.5369	0.4285		0.3265	0.2982	0.2347		0.2463	0.1737	0.1739")

0.7529	0.5369	0.4285		0.3265	0.2982	0.2347		0.2463	0.1737	0.1739


## Removing language entity from question, not from database

In [223]:
q1_eng_pol_heb_nolang = pd.DataFrame(columns=list(embd_data.columns[:3]) + list(embd_data.columns[57:60]) + list(embd_data.columns[66:69]))

In [224]:
id_columns = list(embd_data.columns)
for r in range(embd_np.shape[0]):
    row_data = []
    for col in q1_eng_pol_heb_nolang.columns:
        c = id_columns.index(col)
        row_data.append(calc_cos_sim(ques_eng1_embd[r] - eng_entity, embd_np[r, c]))
    q1_eng_pol_heb_nolang.loc[r] = row_data

In [225]:
q1_eng_pol_heb_nolang[' |'] = ['  ' for _ in range(embd_np.shape[0])]
q1_eng_pol_heb_nolang['| '] = ['  ' for _ in range(embd_np.shape[0])]

In [226]:
q1_eng_pol_heb_nolang = q1_eng_pol_heb_nolang[['English1','Polish1','Hebrew1',' |','English2','Polish2','Hebrew2','| ','English3','Polish3', 'Hebrew3']].round(decimals=4)

In [227]:
q1_eng_pol_heb_nolang

,English1,Polish1,Hebrew1,|,English2,Polish2,Hebrew2,|,English3,Polish3,Hebrew3
0,0.5885,0.4974,0.3656,,0.4611,0.3962,0.1956,,0.3376,0.3224,0.3758
1,0.6492,0.5168,0.2343,,0.4031,0.3522,0.1513,,0.4153,0.3395,0.2236
2,0.8115,0.5815,0.2405,,0.2446,0.1909,0.2263,,0.3472,0.2941,0.1595
3,0.7781,0.5438,0.2751,,0.3951,0.4140,0.1727,,0.6295,0.4840,0.3459
4,0.6200,0.5179,0.3780,,0.4221,0.3279,0.3363,,0.3630,0.2695,0.2816
5,0.6880,0.5466,0.3487,,0.5912,0.4974,0.3083,,0.6224,0.4909,0.2762
6,0.4798,0.3218,0.3402,,0.3176,0.2306,0.2611,,0.1681,0.1743,0.1435
7,0.5278,0.3938,0.2110,,0.3479,0.2530,0.2154,,0.4217,0.3614,0.1394
8,0.7972,0.6054,0.1985,,0.4251,0.3708,0.0440,,0.3791,0.3864,0.1146
9,0.6808,0.4399,0.2814,,0.4474,0.1721,0.1750,,0.4884,0.3601,0.1635


In [178]:
print("0.7144	0.5383	0.4389		0.2663	0.2862	0.2381		0.2015	0.1664	0.1764")

0.7144	0.5383	0.4389		0.2663	0.2862	0.2381		0.2015	0.1664	0.1764
